In [2]:
import matplotlib as mlt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from random import choice


### Итерационный метод Гаусса-Зейделя


In [3]:
def seidel(A, b, eps, x=None):
    L_inv = np.linalg.inv(np.tril(A))
    U = np.triu(A, 1)
    if x is None:
        x = np.zeros(b.shape[0])

    while np.linalg.norm(A @ x - b) > eps:
        x = L_inv @ (b - U @ x)
    return x


### Генератор матриц с диагональным преобладанием


In [4]:
def gen_matrix(n, k):
    a = np.zeros((n, n))
    pos_ij = [0, -1, -2, -3, -4]
    for i in range(n):
        for j in range(n):
            if i == j: continue
            a[i, j] = choice(pos_ij)
        a[i, i] = -np.sum(a[i]) if i > 0 else -np.sum(a[i]) + 10**(-k)
    return a


### Генератор матриц Гильберта


In [5]:
def gen_hilbert(n):
    a = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            a[i, j] = 1 / (i + j + 1)
    return a, a


### Тест


In [6]:
A = np.array(np.mat('16 3; 7 -11'))
b = np.array([11, 13]).T

seidel(A, b, 0.000001)

array([ 0.81218278, -0.66497459])

In [8]:
size = 10
A_k = gen_matrix(size, 0.0001)
F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T
try:
    print(seidel(A_k, F_k, 0.001))
except np.linalg.LinAlgError:
    print('Singular!')

[0.99779704 1.9976454  2.99764617 3.99766892 4.99766508 5.99766806
 6.99766898 7.99769239 8.99770585 9.99767604]



### Deprecated



Попытка сделать на разреженных csr матрицах


def seidel_dep(A, b, eps, x=None):
    L_inv = sparse.linalg.inv(sparse.tril(A))
    U = sparse.triu(A, 1)
    if x is None:
        x = sparse.csr_array(b.shape)

    while sparse.linalg.norm(A @ x - b) > eps:
        x = L_inv @ (b - U @ x)
        print(x.toarray())
    return x

def gen_hilbert_dep(n):
    a = sparse.lil_array((n, n))
    for i in range(n):
        for j in range(n):
            a[i, j] = 1 / (i + j + 1)
    return a.tocsr()

def gen_matrix_dep(n, k):
    a = sparse.lil_array((n, n))
    pos_ij = [0, -1, -2, -3, -4]
    for i in range(n):
        for j in range(n):
            if i == j: continue
            a[i, j] = choice(pos_ij)
        a[i, i] = -a.sum(axis=1)[i] if i > 0 else -a.sum(axis=1)[i] + 10**(-k)
    return a.tocsr()

A_k = gen_matrix_dep(10, 0.0001)
F_k = A_k @ sparse.csr_array(np.arange(1, A_k.shape[0] + 1)).transpose()
#seidel_dep(A_k, F_k, 0.001)

A = sparse.csr_array(np.array(np.mat('16 3; 7 -11')))
b = sparse.csr_array(np.array([11, 13])).transpose()

seidel_dep(A, b, 0.000001).toarray()